# 📘 *2025 Epsom Derby: Pedigree Profiles & Lineage Performance*

Welcome to this exploratory notebook where we dive into the **bloodlines and pedigree data** of the 2025 Epsom Derby runners using a custom-built database of UK and Irish flat racing results from 1990–2025.

> 🧠 **Goal**: To uncover meaningful patterns in **sire-line performance**, assess **lineage strengths over the Derby trip**, and identify which horses might be *bred for greatness*.

---

## 🏇 What We're Exploring

- 📜 **Historical performance** of sires across all distances, with a focus on races **over 1m4f** — the classic Derby distance.  
- 📊 **Pedigree statistics** of this year's entrants, including how well their **sires** and **damsires** have fared in similar conditions.  
- 🔍 A **data-led perspective** to complement traditional form analysis and betting angles.

---

## 💾 Our Tools

- 🐍 **Python + SQLite**: Fast querying of 4.7M+ cleaned race records.  
- 📊 **Pandas + Seaborn**: Clean summaries and visual storytelling.  
- 🧬 **Pedigree metadata**: Including `sire`, `dam`, `damsire`, and race outcomes from 1990 to 2025.

---

## 🔮 Why Pedigree?

The Epsom Derby is unlike any other:

- It's run early in the season over an **unforgiving 1m4f track** with steep cambers.
- Most runners are **unexposed** and stepping up in trip for the first time.
- The right **genetic ingredients** can be the difference between glory and mid-pack anonymity.

---

Let’s dive in and meet the class of 2025...


---

## 🏇 The Preliminary Field: 2025 Epsom Derby

Although final declarations won’t close until five days before the race, the **list of expected runners** for the 2025 Epsom Derby is now widely available via trusted racing sources such as the **Racing Post**.

For major races like the Derby, it’s extremely rare for horses listed at this stage to be withdrawn without injury or unforeseen issues — and any late **supplemented entries** are generally well flagged in advance.

That said, it’s important to understand the difference between **entries**, **declared runners**, and the **final field**:

| Term                  | Meaning                                                                 |
|-----------------------|-------------------------------------------------------------------------|
| 📝 Entries            | Horses entered months in advance to keep the option of running open.    |
| 📋 Declared Runners   | The shortlist of horses officially confirmed 5 days before the race.     |
| 🎬 Final Field        | Usually 12–16 actual starters; some may still be withdrawn or added.     |

> 💡 Fun fact: The Derby allows for **supplementary entries**, where horses not originally entered can be added later — for a hefty fee — if they prove themselves in key trial races.

---

### 🎯 Our Approach

For this analysis, we’re working with the **currently expected field of 42 runners** as listed on the official racecard. These include horses from top yards such as **Aidan O’Brien**, **Charlie Appleby**, **Ralph Beckett**, and more.

We'll:
- Extract their **pedigree data** from our historical database  
- Explore the performance of their **sire lines**  
- Later refine our analysis once the **final declarations** are confirmed closer to race day

Let’s now define this field in code and start pulling their profiles...

---


---

## 🛠️ Step 1: Define the Runner List and Connect to the Database

Before we can analyse the pedigrees or performance of the 2025 Derby field, we need to define the current set of **likely runners** in a format suitable for querying.

To do that, we'll:

- Store the 42 expected runners from the Racing Post racecard in a Python list
- Convert the names to **uppercase**, since the race database stores all horse names in that format
- Establish a connection to our **SQLite database** (`raceform.db`) which contains cleaned UK & Irish flat race records from 1990 to 2025

This sets the foundation for the analysis to come.

---


In [2]:
# List of likely 2025 Epsom Derby runners (as per Racing Post pre-declaration)
# Converted to uppercase to match database formatting

derby_2025_declared = [
    "THE LION IN WINTER", "RULING COURT", "DELACROIX", "TWAIN",
    "MOUNT KILIMANJARO", "LAMBOURN", "GREEN IMPACT", "STANHOPE GARDENS",
    "HOTAZHELL", "DAMYSUS", "OPPORTUNITY", "PRIDE OF ARRAS", "PUPPET MASTER",
    "PUSH THE LIMIT", "SEA SCOUT", "SPANISH VOICE", "TENNESSEE STUD",
    "TRINITY COLLEGE", "ACAPULCO BAY", "LAZY GRIFF", "LUTHER", "PURVIEW",
    "SEACRUISER", "SHACKLETON", "TUSCAN HILLS", "WIMBLEDON HAWKEYE",
    "CALLA LAGOON", "DEVIL'S ADVOCATE", "GREEN STORM", "ISAMBARD BRUNEL",
    "NIGHTIME DANCER", "NIGHTWALKER", "PINHOLE", "ROCK OF CASHEL",
    "ROYAL PLAYWRIGHT", "TORNADO ALERT", "AL WASL STORM", "ROGUE MILLIONS",
    "THRICE", "TOO SOON", "ENEA", "ROGUE IMPACT"
]


In [3]:
import pandas as pd
import sqlite3

# Connect to the SQLite database (using the cleaned table)
db_path = "C:/Users/Rob/Documents/Racing/db/raceform.db"
conn = sqlite3.connect(db_path)


---

## 🔍 Step 2: Extract Race History and Pedigree Fields

To access the pedigree information of each Derby runner, we’ll query our cleaned race database (`data_clean`) for any **past races** involving the 42 likely runners. Even a single run is enough to capture a horse's **sire**, **dam**, and **damsire** — and most of the field has run at least once.

This step also gives us a rich data foundation for **future form analysis**, including:

- 🏟️ Previous racecourses and distances
- 📏 Official and Racing Post Ratings (OR, RPR)
- 🌦️ Track conditions and finishing positions

---

## 🧭 Dealing with Missing Horses: A Hybrid Pedigree Workflow

When we query our database, we expect to retrieve pedigree data for most runners — but not all.

Some horses may be:
- 🐣 **Unraced** — making their debut in the Derby  
- 🌍 **Foreign-trained** or lightly campaigned in jurisdictions outside the database  
- 📆 **Late additions** whose records haven't been captured yet

To ensure a **complete and accurate pedigree table**, we’ll use a **hybrid approach**:

### ✅ Hybrid Workflow Steps:

1. **Query the database first** — extract pedigree info for all runners with past races  
2. **Identify missing horses** — those not found in the database  
3. **Manually look up pedigree details** via trusted sources like Racing Post, Weatherbys, or Equineline  
4. **Append those rows** to our main pedigree dataset for full coverage

> 💡 This approach gives us the best of both worlds: the **efficiency** of structured querying, and the **completeness** of manual supplementation where needed.

---


In [16]:
import difflib

# Step 2: Robust case-insensitive, suffix-tolerant horse name matching

# --- A. Get all unique horse names in DB ---
all_db_horses = pd.read_sql_query("SELECT DISTINCT horse FROM data_clean", conn)
all_db_horses["clean_name"] = all_db_horses["horse"].str.replace(r"\s*\(.*?\)", "", regex=True).str.lower()

# --- B. Clean declared runner names ---
declared_clean = [name.lower() for name in derby_2025_declared]

# --- C. Match declared names to DB horses using fuzzy logic ---
def best_match(name, choices, cutoff=0.9):
    match = difflib.get_close_matches(name, choices, n=1, cutoff=cutoff)
    return match[0] if match else None

# Create mapping: declared -> matched name from DB (if any)
name_map = {}
for name in declared_clean:
    match = best_match(name, all_db_horses["clean_name"].tolist(), cutoff=0.9)
    if match:
        db_name = all_db_horses.loc[all_db_horses["clean_name"] == match, "horse"].values[0]
        name_map[name] = db_name

# --- D. Pull matched race records ---
matched_names = list(name_map.values())
race_history_df = pd.read_sql_query(
    f"""
    SELECT horse, sire, dam, damsire, trainer, date, course, race_name, dist, pos, rpr, ts, going
    FROM data_clean
    WHERE horse IN ({','.join(['?'] * len(matched_names))})
    ORDER BY horse, date DESC
    """,
    conn,
    params=matched_names
)

# --- E. Find which runners were NOT matched ---
found_horses = set(name_map.keys())
not_found_horses = [name for name in declared_clean if name not in found_horses]

# Optional: recover original casing for display
name_lookup = {n.lower(): n for n in derby_2025_declared}
not_found_horses_display = [name_lookup[name] for name in not_found_horses]

# Summary
print(f"✅ Horses found in DB: {len(found_horses)}")
print(f"⚠️ Horses not found in DB: {len(not_found_horses_display)}")
print("🔍 Missing horses:")
for horse in not_found_horses_display:
    print("-", horse)


✅ Horses found in DB: 42
⚠️ Horses not found in DB: 0
🔍 Missing horses:


In [9]:
# Step 3: Placeholder for manually added pedigree records
manual_pedigree_df = pd.DataFrame(columns=["horse", "sire", "dam", "damsire"])

---

## 🧬 Step 3: Build One Pedigree Record per Runner

Now that we’ve successfully retrieved complete race history for all 42 declared runners, we can extract a **single pedigree profile** for each.

Pedigree fields (`sire`, `dam`, `damsire`) do not change over a horse’s career, so we simply take the most recent available record for each horse from the race history.

This creates a clean and consistent `pedigree_df` that we will use in all downstream analysis.

---



In [17]:
# Step 3: Build one pedigree row per horse from latest race

latest_pedigree = race_history_df.sort_values("date", ascending=False).drop_duplicates(subset="horse")
pedigree_df = latest_pedigree[["horse", "sire", "dam", "damsire"]].sort_values("horse").reset_index(drop=True)

# Show preview
pedigree_df.head()



,horse,sire,dam,damsire
0,Acapulco Bay (GB),Pursuit Of Love (GB),Lapu-Lapu GB,Prince Sabo
1,Al Wasl Storm (IRE),Affinisea (IRE),Dearborn (FR),Martaline
2,Calla Lagoon (GB),New Bay (GB),Orchid Lily GB,Dansili
3,DEVIL'S ADVOCATE,MISTER MAT,Ecyba,CYBORG
4,Damysus (GB),Frankel (GB),Legerete (USA),Rahy


---

## 📊 Step 4: Historical Performance of Sires

Now that we’ve extracted pedigree info for each runner, we’ll analyse how their **sires have performed historically** across the entire race database.

For each sire, we calculate:

- 🧮 Total number of runs by their progeny
- 🏁 Average finishing position (excluding fallers/PU/etc.)
- 🥇 Win count and 🥈 top-3 place count
- 📏 Number of runs over **12 furlongs or more** — key for assessing Derby stamina

This helps us evaluate which runners may have the strongest **genetic profile for Classic distances**.

---


In [18]:
# Step 4: Analyse historical performance of these sires

# Get the list of sires from pedigree_df
sire_list = pedigree_df["sire"].dropna().unique().tolist()

# Query database to get aggregate stats for each sire's progeny
# reminder pos > 40 = DNF
query = f"""
SELECT sire,
       COUNT(*) AS total_runs,
       AVG(CASE WHEN pos < 40 THEN pos ELSE NULL END) AS avg_position,
       SUM(CASE WHEN pos = 1 THEN 1 ELSE 0 END) AS total_wins,
       SUM(CASE WHEN pos <= 3 THEN 1 ELSE 0 END) AS total_places,
       SUM(CASE WHEN dist >= 12 THEN 1 ELSE 0 END) AS runs_12f_plus
FROM data_clean
WHERE sire IN ({','.join(['?'] * len(sire_list))})
GROUP BY sire
"""

sire_stats_df = pd.read_sql_query(query, conn, params=sire_list)

# Merge back into the pedigree_df so we can see each runner with their sire stats
pedigree_summary = pedigree_df.merge(sire_stats_df, on="sire", how="left")

# Display results
pedigree_summary.sort_values("sire")


,horse,sire,dam,damsire,total_runs,avg_position,total_wins,total_places,runs_12f_plus
1,Al Wasl Storm (IRE),Affinisea (IRE),Dearborn (FR),Martaline,486,5.051440,75,210,486
23,Rock Of Cashel (GER),Areion (GER),Rocket Light (GER),Lando,1352,6.143491,135,384,1352
11,Lambourn (IRE),Australia (GB),Gossamer Wings (USA),Scat Daddy,4254,5.637047,501,1383,4254
20,Puppet Master (IRE),Camelot (GB),Realtra (IRE),Dark Angel,6174,5.660674,725,2117,6174
30,Shackleton (IRE),Camelot (GB),Damaniyat Girl (USA),Elusive Quality,6174,5.660674,725,2117,6174
36,Too Soon (AUS),Charge Forward (AUS),Canny Lassie (AUS),Canny Lad,257,7.194553,15,40,257
8,Green Storm (IRE),Circus Maximus (IRE),Banimpire (IRE),Holy Roman Emperor,48,6.312500,4,16,48
38,Trinity College (IRE),Dubawi (IRE),Hermosa (IRE),Galileo,10319,5.111929,1824,4289,10319
5,Delacroix (IRE),Dubawi (IRE),Tepin (USA),Bernstein,10319,5.111929,1824,4289,10319
25,Rogue Millions (GB),Dubawi (IRE),Nyaleti (IRE),Arch,10319,5.111929,1824,4289,10319


---

## 📘 Step 5: Pedigree Performance Summary — Stamina Bias & Sire Quality

Now that we've built a complete pedigree dataset, it's time to focus on the key question:

> **Which Derby runners are *bred* to stay the 1m4f trip — and which might not be?**

The Epsom Derby is an unforgiving test of stamina, balance, and class. While form analysis is useful, many classic contenders are lightly raced or unexposed — so pedigree can offer crucial early clues.

---

### 🎯 Why This Step Matters

Sires vary enormously in the distance preferences and success rates of their progeny. By calculating **historical performance metrics** across their offspring, we can uncover:

- ⛽ **Stamina Bias** — does this sire typically produce stayers?
- 🏁 **Win & Place Rates** — how often do they finish well?
- 📉 **Average Finishing Position** — a sanity check on overall performance

---

### 📊 What We’re Measuring

For each runner’s **sire**, we compute:

| Metric        | Description                                                  |
|---------------|--------------------------------------------------------------|
| `win_pct`     | Win percentage: total_wins / total_runs                      |
| `place_pct`   | Place percentage (Top 3): total_places / total_runs          |
| `stamina_bias`| Long-distance bias: 12f+ runs / total_runs                   |
| `avg_position`| Mean finishing position across all distances                 |

This will allow us to sort, visualise, and tag each runner based on **genetic suitability for the Derby test**.

--- 


In [19]:
# Step 5: Calculate derived performance metrics for each sire
sire_stats_df["win_pct"] = sire_stats_df["total_wins"] / sire_stats_df["total_runs"]
sire_stats_df["place_pct"] = sire_stats_df["total_places"] / sire_stats_df["total_runs"]
sire_stats_df["stamina_bias"] = sire_stats_df["runs_12f_plus"] / sire_stats_df["total_runs"]

# Optional: Round for readability
sire_stats_df[["win_pct", "place_pct", "stamina_bias"]] = sire_stats_df[["win_pct", "place_pct", "stamina_bias"]].round(3)

# Merge back into pedigree_df using 'sire' as the key
pedigree_summary_df = pedigree_df.merge(sire_stats_df, on="sire", how="left")

# Preview
pedigree_summary_df.sort_values("stamina_bias", ascending=False).head()


,horse,sire,dam,damsire,total_runs,avg_position,total_wins,total_places,runs_12f_plus,win_pct,place_pct,stamina_bias
0,Acapulco Bay (GB),Pursuit Of Love (GB),Lapu-Lapu GB,Prince Sabo,61,4.426230,8,27,61,0.131,0.443,1.0
31,Spanish Voice (GB),Lope De Vega (IRE),Criteria (IRE),Galileo,10928,5.454978,1457,4011,10928,0.133,0.367,1.0
23,Rock Of Cashel (GER),Areion (GER),Rocket Light (GER),Lando,1352,6.143491,135,384,1352,0.100,0.284,1.0
24,Rogue Impact (GB),Study Of Man (IRE),Just So (IRE),Galileo,398,4.949749,64,161,398,0.161,0.405,1.0
25,Rogue Millions (GB),Dubawi (IRE),Nyaleti (IRE),Arch,10319,5.111929,1824,4289,10319,0.177,0.416,1.0


---

## 📘 Step 6: Ranked Pedigree Table — Which Runners Are Bred for the Derby?

With a complete pedigree dataset and performance metrics for all sires, we now distill that information into a clear, ranked table.

> 🧬 **Our goal:** Identify runners whose sire lines have produced top-class stayers — and flag those that may struggle over the Derby trip.

---

### 🔍 What We’re Measuring

For each runner, we rank by:
- `win_pct` — How often the sire's offspring win
- `stamina_bias` — What % of their runs are over 12f+
- `avg_position` — Their average finishing position

We combine these into a simple **pedigree verdict**:
- ✅ **Strong** — Classic-producing sires with staying power
- ⚠️ **Mixed** — Useful lines with a few question marks
- ❌ **Risky** — Low win rates, poor average placings, or stamina doubts

---

### 🏇 Why It Matters

This lets us:
- Quickly highlight likely stayers and hidden pedigree value
- Begin narrowing the field based on deep bloodline insight
- Create a foundation for form and trainer analysis later

Now let’s display the table and highlight the runners bred for greatness.

---


In [25]:
# Step 6: Ranked Pedigree Table — Sire Performance Scoring

# Select relevant columns
pedigree_ranking_df = pedigree_summary_df[[
    "horse", "sire", "win_pct", "stamina_bias", "avg_position"
]].copy()

# Round for display
pedigree_ranking_df["win_pct"] = (pedigree_ranking_df["win_pct"] * 100).round(1)
pedigree_ranking_df["stamina_bias"] = pedigree_ranking_df["stamina_bias"].round(2)
pedigree_ranking_df["avg_position"] = pedigree_ranking_df["avg_position"].round(2)

# Add pedigree verdict
def pedigree_verdict(row):
    if row["win_pct"] >= 14 and row["stamina_bias"] >= 0.9 and row["avg_position"] <= 5.5:
        return "✅ Strong"
    elif row["win_pct"] <= 10 or row["avg_position"] > 6.0:
        return "❌ Risky"
    else:
        return "⚠️ Mixed"

pedigree_ranking_df["pedigree_verdict"] = pedigree_ranking_df.apply(pedigree_verdict, axis=1)

# Sort: strong → mixed → risky, then by win rate
pedigree_ranking_df = pedigree_ranking_df.sort_values(
    by=["pedigree_verdict", "win_pct", "stamina_bias"],
    ascending=[True, False, False]
)

# Display the final ranked pedigree table
pedigree_ranking_df.style.bar(subset=["win_pct", "stamina_bias"], color="#dff0d8").set_caption("Derby Pedigree Rankings")


,horse,sire,win_pct,stamina_bias,avg_position,pedigree_verdict
15,Nightime Dancer (IRE),Ghaiyyath (IRE),13.900000,1.000000,4.680000,⚠️ Mixed
32,Stanhope Gardens (IRE),Ghaiyyath (IRE),13.900000,1.000000,4.680000,⚠️ Mixed
10,Isambard Brunel (USA),Justify (USA),13.300000,1.000000,5.450000,⚠️ Mixed
26,Royal Playwright (GB),Lope De Vega (IRE),13.300000,1.000000,5.450000,⚠️ Mixed
27,Ruling Court (USA),Justify (USA),13.300000,1.000000,5.450000,⚠️ Mixed
31,Spanish Voice (GB),Lope De Vega (IRE),13.300000,1.000000,5.450000,⚠️ Mixed
0,Acapulco Bay (GB),Pursuit Of Love (GB),13.100000,1.000000,4.430000,⚠️ Mixed
14,Mount Kilimanjaro (FR),Siyouni (FR),13.000000,1.000000,5.600000,⚠️ Mixed
28,Sea Scout (GB),Lomitas (GB),13.000000,1.000000,5.290000,⚠️ Mixed
22,Push The Limit (FR),Le Havre (IRE),12.500000,1.000000,5.850000,⚠️ Mixed
